In [135]:
# https://stackoverflow.com/a/3941967/62997
# https://gist.github.com/mbarkhau/afad5ea4d3640d58df50251b449e49d7
def mk_primes(limit):
    a = [True] * limit                          # Initialize the primality list
    a[0] = a[1] = False

    for i, isprime in enumerate(a):
        if isprime:
            yield i
            for n in range(i*i, limit, i):     # Mark factors non-prime
                a[n] = False

In [136]:
from math import sqrt
from functools import lru_cache, reduce
from collections import Counter
from itertools import product
 
 
MUL = int.__mul__
 
 
def prime_factors(n):
    'Map prime factors to their multiplicity for n'
    d = _divs(n)
    d = [] if d == [n] else (d[:-1] if d[-1] == d else d)
    pf = Counter(d)
    return dict(pf)
 
@lru_cache(maxsize=None)
def _divs(n):
    'Memoized recursive function returning prime factors of n as a list'
    for i in range(2, int(sqrt(n)+1)):
        d, m  = divmod(n, i)
        if not m:
            return [i] + _divs(d)
    return [n]
 
 
def proper_divs(n):
    '''Return the set of proper divisors of n.'''
    pf = prime_factors(n)
    pfactors, occurrences = pf.keys(), pf.values()
    multiplicities = product(*(range(oc + 1) for oc in occurrences))
    divs = {reduce(MUL, (pf**m for pf, m in zip(pfactors, multis)), 1)
            for multis in multiplicities}
    try:
        divs.remove(n)
    except KeyError:
        pass
    return divs or ({1} if n != 1 else set())
 

In [137]:
for i in range(100): 
    ours = proper_divisors(i)
    theirs = proper_divs(i)
    if ours != theirs: 
        print(ours, theirs)
        print(i)

{1, 2, 10, 5} {1, 2, 4, 5, 10}
20
{1, 2, 14, 7} {1, 2, 4, 7, 14}
28
{1, 2, 3, 7, 14, 21} {1, 2, 3, 6, 7, 14, 21}
42
{1, 2, 11, 22} {1, 2, 4, 11, 22}
44
{1, 2, 26, 13} {1, 2, 4, 13, 26}
52
{1, 2, 3, 33, 11, 22} {1, 2, 3, 33, 6, 11, 22}
66
{1, 2, 34, 17} {1, 2, 34, 4, 17}
68
{1, 2, 19, 38} {1, 2, 4, 38, 19}
76
{1, 2, 3, 39, 13, 26} {1, 2, 3, 6, 39, 13, 26}
78
{1, 2, 11, 44, 22} {1, 2, 4, 8, 11, 44, 22}
88
{1, 2, 46, 23} {1, 2, 4, 46, 23}
92
{11, 1, 3, 33} {1, 33, 3, 9, 11}
99


### Non-abundant sums

#### Problem 23



A perfect number is a number for which the sum of its proper divisors is exactly equal to the number. For example, the sum of the proper divisors of 28 would be 1 + 2 + 4 + 7 + 14 = 28, which means that 28 is a perfect number.

A number n is called deficient if the sum of its proper divisors is less than n and it is called abundant if this sum exceeds n.

As 12 is the smallest abundant number, 1 + 2 + 3 + 4 + 6 = 16, the smallest number that can be written as the sum of two abundant numbers is 24. By mathematical analysis, it can be shown that all integers greater than 28123 can be written as the sum of two abundant numbers. However, this upper limit cannot be reduced any further by analysis even though it is known that the greatest number that cannot be expressed as the sum of two abundant numbers is less than this limit.

Find the sum of all the positive integers which cannot be written as the sum of two abundant numbers.


In [138]:
def is_abundant(n): 
    return n < sum(proper_divs(n))

In [139]:
prime_list = list(mk_primes(100))
prime_list = list(mk_primes(100000))
prime_set = set(prime_list)
proper_divisors(18)

{1, 2, 3, 6, 9}

In [140]:
is_abundant(18)

True

In [141]:
limit = 28123
def abundant_numbers(limit):
    return [n for n in range(1, limit + 1) if is_abundant(n)]

In [142]:
from itertools import combinations_with_replacement

def sum_of_abundand_numbers():
    
    abund_nums = abundant_numbers(2 * limit)
    #new_limit = abund_nums[-1] * 2
     
    # print(new_limit)
    
    candidates = set(range(1, 2 * limit + 1))
    
    pair_sums = set([a+b for (a,b) in combinations_with_replacement(abund_nums, 2)])
    #print(list(pairs)[:100])
    x = candidates - pair_sums
    #print(sorted(list(x)))
    return sum(x)  

In [143]:
sum_of_abundand_numbers()

4179871

In [144]:
def other_approach():
    s = 0
    candidates = set(range(1, limit + 1))
    abund_nums = set(abundant_numbers(limit))
    good_candidates = []
    for candidate in candidates:
        good_candidate = True 
        for an in abund_nums:
            if (candidate - an) in abund_nums: 
                good_candidate = False
                break 
        if good_candidate:
            good_candidates += [candidate] 
    return sum(good_candidates)

In [145]:
other_approach()

4179871

### Amicable numbers
#### Problem 21



Let d(n) be defined as the sum of proper divisors of n (numbers less than n which divide evenly into n).
If d(a) = b and d(b) = a, where a ≠ b, then a and b are an amicable pair and each of a and b are called amicable numbers.

For example, the proper divisors of 220 are 1, 2, 4, 5, 10, 11, 20, 22, 44, 55 and 110; therefore d(220) = 284. The proper divisors of 284 are 1, 2, 4, 71 and 142; so d(284) = 220.

Evaluate the sum of all the amicable numbers under 10000.


In [150]:
mdict = {}
limit = 10000
for i in range(limit): 
    mdict[i] = sum(proper_divs(i))
    
s = 0
for k, v in mdict.items():
    if k == v:
        continue 
    if v in mdict and mdict[v] == k:
        s += v
        
print(mdict[220])
print(mdict[284])
print(s)

s1 = 0
for n in range(10000):
    sod = sum(proper_divs(n))
    sodsod = sum(proper_divs(sod))
    if sodsod == n and sod != n: 
        print(n)
        s1 += n
print(s1)

284
220
31627
0
1
220
284
1184
1210
2620
2924
5020
5564
6232
6368
31627


In [152]:
proper_divs(0)

{1}

### Largest product in a grid
Problem 11

In the 20×20 grid below, four numbers along a diagonal line have been marked in red.

08 02 22 97 38 15 00 40 00 75 04 05 07 78 52 12 50 77 91 08
49 49 99 40 17 81 18 57 60 87 17 40 98 43 69 48 04 56 62 00
81 49 31 73 55 79 14 29 93 71 40 67 53 88 30 03 49 13 36 65
52 70 95 23 04 60 11 42 69 24 68 56 01 32 56 71 37 02 36 91
22 31 16 71 51 67 63 89 41 92 36 54 22 40 40 28 66 33 13 80
24 47 32 60 99 03 45 02 44 75 33 53 78 36 84 20 35 17 12 50
32 98 81 28 64 23 67 10 26 38 40 67 59 54 70 66 18 38 64 70
67 26 20 68 02 62 12 20 95 63 94 39 63 08 40 91 66 49 94 21
24 55 58 05 66 73 99 26 97 17 78 78 96 83 14 88 34 89 63 72
21 36 23 09 75 00 76 44 20 45 35 14 00 61 33 97 34 31 33 95
78 17 53 28 22 75 31 67 15 94 03 80 04 62 16 14 09 53 56 92
16 39 05 42 96 35 31 47 55 58 88 24 00 17 54 24 36 29 85 57
86 56 00 48 35 71 89 07 05 44 44 37 44 60 21 58 51 54 17 58
19 80 81 68 05 94 47 69 28 73 92 13 86 52 17 77 04 89 55 40
04 52 08 83 97 35 99 16 07 97 57 32 16 26 26 79 33 27 98 66
88 36 68 87 57 62 20 72 03 46 33 67 46 55 12 32 63 93 53 69
04 42 16 73 38 25 39 11 24 94 72 18 08 46 29 32 40 62 76 36
20 69 36 41 72 30 23 88 34 62 99 69 82 67 59 85 74 04 36 16
20 73 35 29 78 31 90 01 74 31 49 71 48 86 81 16 23 57 05 54
01 70 54 71 83 51 54 69 16 92 33 48 61 43 52 01 89 19 67 48

The product of these numbers is 26 × 63 × 78 × 14 = 1788696.

What is the greatest product of four adjacent numbers in the same direction (up, down, left, right, or diagonally) in the 20×20 grid?


In [153]:
mt = [
    [ 8,  2, 22, 97, 38, 15,  0, 40,  0, 75,  4,  5,  7, 78, 52, 12, 50, 77, 91,  8],
    [49, 49, 99, 40, 17, 81, 18, 57, 60, 87, 17, 40, 98, 43, 69, 48,  4, 56, 62,  0],
    [81, 49, 31, 73, 55, 79, 14, 29, 93, 71, 40, 67, 53, 88, 30,  3, 49, 13, 36, 65],
    [52, 70, 95, 23,  4, 60, 11, 42, 69, 24, 68, 56,  1, 32, 56, 71, 37,  2, 36, 91],
    [22, 31, 16, 71, 51, 67, 63, 89, 41, 92, 36, 54, 22, 40, 40, 28, 66, 33, 13, 80],
    [24, 47, 32, 60, 99,  3, 45,  2, 44, 75, 33, 53, 78, 36, 84, 20, 35, 17, 12, 50],
    [32, 98, 81, 28, 64, 23, 67, 10, 26, 38, 40, 67, 59, 54, 70, 66, 18, 38, 64, 70],
    [67, 26, 20, 68,  2, 62, 12, 20, 95, 63, 94, 39, 63,  8, 40, 91, 66, 49, 94, 21],
    [24, 55, 58,  5, 66, 73, 99, 26, 97, 17, 78, 78, 96, 83, 14, 88, 34, 89, 63, 72],
    [21, 36, 23,  9, 75,  0, 76, 44, 20, 45, 35, 14,  0, 61, 33, 97, 34, 31, 33, 95],
    [78, 17, 53, 28, 22, 75, 31, 67, 15, 94,  3, 80,  4, 62, 16, 14,  9, 53, 56, 92],
    [16, 39,  5, 42, 96, 35, 31, 47, 55, 58, 88, 24,  0, 17, 54, 24, 36, 29, 85, 57],
    [86, 56,  0, 48, 35, 71, 89,  7,  5, 44, 44, 37, 44, 60, 21, 58, 51, 54, 17, 58],
    [19, 80, 81, 68,  5, 94, 47, 69, 28, 73, 92, 13, 86, 52, 17, 77,  4, 89, 55, 40],
    [ 4, 52,  8, 83, 97, 35, 99, 16,  7, 97, 57, 32, 16, 26, 26, 79, 33, 27, 98, 66],
    [88, 36, 68, 87, 57, 62, 20, 72,  3, 46, 33, 67, 46, 55, 12, 32, 63, 93, 53, 69],
    [ 4, 42, 16, 73, 38, 25, 39, 11, 24, 94, 72, 18,  8, 46, 29, 32, 40, 62, 76, 36],
    [20, 69, 36, 41, 72, 30, 23, 88, 34, 62, 99, 69, 82, 67, 59, 85, 74,  4, 36, 16],
    [20, 73, 35, 29, 78, 31, 90,  1, 74, 31, 49, 71, 48, 86, 81, 16, 23, 57,  5, 54],
    [ 1, 70, 54, 71, 83, 51, 54, 69, 16, 92, 33, 48, 61, 43, 52,  1, 89, 19, 67, 48],
]

In [155]:
n = len(mt)
max_so_far = 0

for i in range(n):
    for j in range(n):
        right = 0
        down = 0
        down_right = 0
        down_left = 0
        
        if j < n - 4: 
            right = mt[i][j] * mt[i][j+1] * mt[i][j+2] * mt[i][j+3]
        if i < n - 4:
            down = mt[i][j] * mt[i+1][j] * mt[i+2][j] * mt[i+3][j] 
        if i < n - 4 and j < n - 4:
            down_right = mt[i][j] * mt[i+1][j+1] * mt[i+2][j+2] * mt[i+3][j+3]        
        if i < n - 4 and j > 4:
            down_left = mt[i][j] * mt[i+1][j-1] * mt[i+2][j-2] * mt[i+3][j-3]
        m = max([right,down, down_right, down_left])
        if max_so_far < m:
            max_so_far = m

print(max_so_far)
        

70600674


### Factorial digit sum
Problem 20

n! means n × (n − 1) × ... × 3 × 2 × 1

For example, 10! = 10 × 9 × ... × 3 × 2 × 1 = 3628800,
and the sum of the digits in the number 10! is 3 + 6 + 2 + 8 + 8 + 0 + 0 = 27.

Find the sum of the digits in the number 100!


In [ ]:
M did it

### Problem 2

In [157]:
import functools 

@functools.lru_cache(1000)
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

In [160]:
fib(60)

1548008755920

In [161]:
sum(fib(n) for n in range(60) if fib(n) < 4_000_000 and fib(n) % 2 == 0)

4613732